# Coursera Applied Data Science Capstone - Week 3

## Part 1

First we import the necessary packages. We import 'BeautifulSoup' to scrape and parse our data, 'requests' to be able to retrieve the data on Wikipedia, 'pandas' to put our scraped data into a dataframe, and 'numpy' to perform manipulation on our strings.

In [224]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

We retrieve the html text from the wiki page, and put it into a "soup" object as 'lxml' using BeautifulSoup

In [225]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'lxml')

Looking through the source code shows us that all the text we need is within 'td' html. We assign all the 'td' sections to a "table_data" object. Then we create an empty list called "postcodes", and for every entry in "table_data" up to the 865th entry, we return the text within the 'td' html, and add it to the "postcodes" list. I admittedly found it to be the first 865 entries through brute force.

In [226]:
table_data = soup.find_all('td')
postcodes=[]
for i in table_data[0:864]:
    postcodes.append(i.text)

In our "postcodes" list, every 1st entry is a postal code, every 2nd entry is a borough, and every 3rd entry is a neighborhood. We create a dataframe to contain these, and slice it accordingly so that each column contains the correct data. This lists the columns alphabetically by default, so we need an extra line of code to make sure the columns are in the order we want.

In [227]:
df = pd.DataFrame({'Postal Code': postcodes[0::3], 'Borough': postcodes[1::3], 'Neighborhood': postcodes[2::3]})
df = df[['Postal Code','Borough','Neighborhood']]
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


We don't want the '\n' in our Neighborhood columns, so we delete them (by replacing them with nothing). We also change 'Not assigned' boroughs to NaN so that we can then delete the rows containing them.

In [228]:
df.Neighborhood = df.Neighborhood.str.replace('\n','')
df.Borough = df.Borough.replace('Not assigned', np.nan)
df = df.dropna()
df.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Now let's combine rows where the postal codes are the same.

In [229]:
df = df.groupby(['Postal Code','Borough'], as_index=False, sort=False).agg(','.join)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Finally, if a neighborhood is 'Not assigned', we replace it with its respective borough name.

In [230]:
df.Neighborhood.replace('Not assigned', df.Borough, inplace=True)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


How many rows and columns does our dataframe have?

In [231]:
df.shape

(103, 3)

## Part 2

We use 'pandas' to read the csv from the url given on Coursera, and have a look at the csv dataframe.

In [232]:
geo_co = pd.read_csv('https://cocl.us/Geospatial_data')

In [233]:
geo_co.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's merge the 'geo_co' and 'df' dataframes.

In [234]:
df = pd.merge(df, geo_co, on='Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


## Part 3

Now we will explore and cluster the neighborhoods in Toronto. Let's only use boroughs that contain the word Toronto to make things a bit simpler.

In [235]:
df = df.loc[df['Borough'].str.contains('Toronto')]
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
36,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


How many boroughs are we going to examine?

In [236]:
df.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

Let's import some extra packages that we'll need to map and cluster our neighborhoods.

In [237]:
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


First we need to map "Toronto, Ontario" in general. We will name the user_agent *toronto_explorer*.

In [238]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Now we map Toronto, and then superimpose our neighborhoods of interest on top.

In [239]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

To start finding venues in our neighborhoods, we will need to import another library, and retrieve our FourSquare API information.

In [240]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [241]:
CLIENT_ID = 'H2MK3PRFQFIN1RCJJ0DYDQHSSCLLSFHGGYOUT3MNTEJVIWOQ'
CLIENT_SECRET = '0MNEQJKP1CIO3UOHPGY4AODF4DNBUSOHOH4DABVDVS0Y0J2Y'
VERSION = '20180605'

Next we define a function that retrieves the category of a venue.

In [242]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we define a function that retrieves up to 100 venues within 500 meters of the coordinates of each neighborhood.

In [243]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now using this function, we will create a new dataframe called *toronto_venues*.

In [244]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [245]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


How many categories of venues do we have?

In [246]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


Now we shall use **one hot encoding** to further analyze our data.

In [247]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# put neighborhood column at front of dataframe
tor_neigh = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', tor_neigh)

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [248]:
toronto_onehot.shape

(1721, 236)

The above code tells us that we have a total of 1721 venues, over 236 unique categories.

Now let's group by neighborhood and observe the mean of frequency of occurrence of each category.

In [249]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.020000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011628,...,0.000000,0.000000,0.00,0.011628,0.000000,0.000000,0.011628,0.00,0.000000,0.011628
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.010000,0.000000,0.00,0.060000,0.000000,0.050000,0.010000,0.00,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,Church and Wellesley,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011236,...,0.000000,0.000000,0.00,0.000000,0.011236,0.011236,0.000000,0.00,0.011236,0.022472


We want to know the most frequent venues of each neighborhood. We start by defining a function to sort the venues in descending order.

In [250]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then we create a dataframe that displays the top 10 venues for each neighborhood.

In [251]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Sushi Restaurant,Asian Restaurant,Bakery,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Grocery Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Park,Smoke Shop,Skate Park,Brewery,Burrito Place,Restaurant,Recording Studio,Comic Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport Food Court


Let's run *k-means* to cluster the neighborhoods into 4 different clusters.

In [252]:
# drop the Neighborhood column so it doesn't interfere with clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = 4, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3], dtype=int32)

We create a dataframe that includes cluster labels, and the top 10 common venues.

In [253]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,3,Coffee Shop,Café,Park,Pub,Bakery,Mexican Restaurant,Restaurant,Theater,Breakfast Spot,Brewery
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Pizza Place,Sporting Goods Shop,Hotel,Italian Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Hotel,Café,Restaurant,Cosmetics Shop,Italian Restaurant,Cocktail Bar,Gastropub,Breakfast Spot,Bakery
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Coffee Shop,Health Food Store,Pub,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Restaurant


Now visualize the resulting clusters.

In [254]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

Let's see what's most popular in each of our clusters and then name them appropriately.

In [255]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_1.groupby(['1st Most Common Venue']).size()

1st Most Common Venue
Park    3
dtype: int64

*cluster_1* will be our Park cluster.

In [256]:
cluster_park = cluster_1

cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_2.groupby(['1st Most Common Venue']).size()

1st Most Common Venue
Garden    1
dtype: int64

*cluster_2* will be our Garden cluster.

In [257]:
cluster_garden = cluster_2

cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_3.groupby(['1st Most Common Venue']).size()

1st Most Common Venue
Playground    1
dtype: int64

*cluster_3* will be our Playground cluster.

In [258]:
cluster_playground = cluster_3

cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_4.groupby(['1st Most Common Venue']).size()

1st Most Common Venue
Airport Service         1
Bar                     1
Breakfast Spot          2
Café                    5
Clothing Store          1
Coffee Shop            15
Greek Restaurant        1
Grocery Store           1
Gym                     1
Japanese Restaurant     1
Light Rail Station      1
Park                    1
Sandwich Place          1
Supermarket             1
dtype: int64

*cluster_4* will be our Coffee Shop cluster.

In [259]:
cluster_coffee_shop = cluster_4

Finally we can look at our clusters in detail.

In [260]:
cluster_park

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
68,Central Toronto,0,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
91,Downtown Toronto,0,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [261]:
cluster_playground

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,2,Playground,Gym,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [262]:
cluster_garden

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [263]:
cluster_coffee_shop

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Café,Park,Pub,Bakery,Mexican Restaurant,Restaurant,Theater,Breakfast Spot,Brewery
9,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Pizza Place,Sporting Goods Shop,Hotel,Italian Restaurant
15,Downtown Toronto,3,Coffee Shop,Hotel,Café,Restaurant,Cosmetics Shop,Italian Restaurant,Cocktail Bar,Gastropub,Breakfast Spot,Bakery
19,East Toronto,3,Coffee Shop,Health Food Store,Pub,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
20,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Restaurant
24,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Café,Ice Cream Shop,Sushi Restaurant,Japanese Restaurant,Spa
25,Downtown Toronto,3,Grocery Store,Café,Park,Baby Store,Athletics & Sports,Diner,Restaurant,Nightclub,Italian Restaurant,Coffee Shop
30,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Sushi Restaurant,Asian Restaurant,Bakery,Bar
31,West Toronto,3,Supermarket,Pharmacy,Bakery,Pizza Place,Music Venue,Pool,Middle Eastern Restaurant,Café,Discount Store,Brewery
36,Downtown Toronto,3,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Pizza Place,Brewery,Bakery,Scenic Lookout


### This concludes the assignment.